In [3]:
import re
import requests
import json
from collections import defaultdict
import copy
import pandas as pd
import uuid
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import psycopg2

In [ ]:
# # the base URLs with a placeholder state (WV)
# base_urls = {
#     "rental_listings": (
#         "https://api.rentcast.io/v1/listings/rental/long-term"
#         "?state=WV&status=Active&daysOld=5&limit=500"
#     ),
#     "sale_listings": (
#         "https://api.rentcast.io/v1/listings/sale"
#         "?state=WV&status=Active&daysOld=5&limit=500"
#     )
# }

# states = ["VA", "SC", "NC", "MD", "GA", "FL", "DE"]

# load_dotenv()  # giving the API_KEY to os.environ

# API_KEY = os.getenv("API_KEY")
# if not API_KEY:
#     raise RuntimeError("API_KEY not set in environment")

# headers = {
#     "accept": "application/json",
#     "X-Api-Key": API_KEY
# }

# # Initialize an empty dict to hold results
# all_data = {}
# for region in base_urls:
#     all_data[region] = {}

# # Loop through each region and each state, swapping out the state code
# for region, url_template in base_urls.items():
    
#     # First handle the template state "WV"
#     state_code = "WV"
#     url = re.sub(r"state=[A-Z]{2}", f"state={state_code}", url_template)
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         all_data[region][state_code] = response.json()
#     else:
#         all_data[region][state_code] = None
#         print(f"Error fetching {region} for {state_code}: {response.status_code}")

#     # looping through the rest of the states
#     for state_code in states:
#         url = re.sub(r"state=[A-Z]{2}", f"state={state_code}", url_template)
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             all_data[region][state_code] = response.json()
#         else:
#             all_data[region][state_code] = None
#             print(f"Error fetching {region} for {state_code}: {response.status_code}")


[{'id': '1009-Meadow-Glen-Cir,-Fairburn,-GA-30213', 'formattedAddress': '1009 Meadow Glen Cir, Fairburn, GA 30213', 'addressLine1': '1009 Meadow Glen Cir', 'addressLine2': None, 'city': 'Fairburn', 'state': 'GA', 'zipCode': '30213', 'county': 'Fulton', 'latitude': 33.534457, 'longitude': -84.58188, 'propertyType': 'Single Family', 'bedrooms': 4, 'bathrooms': 2, 'squareFootage': 1863, 'lotSize': 5968, 'yearBuilt': 1999, 'status': 'Active', 'price': 2135, 'listingType': 'Standard', 'listedDate': '2025-06-25T00:00:00.000Z', 'removedDate': None, 'createdDate': '2023-02-28T00:00:00.000Z', 'lastSeenDate': '2025-06-25T04:41:33.784Z', 'daysOnMarket': 1, 'history': {'2025-05-03': {'event': 'Rental Listing', 'price': 2249, 'listingType': 'Standard', 'listedDate': '2025-05-03T00:00:00.000Z', 'removedDate': '2025-05-04T00:00:00.000Z', 'daysOnMarket': 1}, '2025-05-18': {'event': 'Rental Listing', 'price': 2210, 'listingType': 'Standard', 'listedDate': '2025-05-18T00:00:00.000Z', 'removedDate': '202

In [4]:

# print(json.dumps(rental_listings, indent=2))
with open('all_data.json', 'r', encoding='utf-8') as f:
    all_data = json.load(f)

In [5]:
# Extracting the 2 main data endpoints from the global data
sale_listings = all_data["sale_listings"]
rental_listings = all_data["rental_listings"]

In [6]:
# saving the jason data to current directory
with open('sale_listings.json', 'w', encoding='utf-8') as f:
    json.dump(sale_listings, f, ensure_ascii=False, indent=2)


with open('rental_listings.json', 'w', encoding='utf-8') as f:
    json.dump(rental_listings, f, ensure_ascii=False, indent=2)

In [7]:
# Making a copy of the data before editing
sale_listings_copy = copy.deepcopy(sale_listings)
rental_listings_copy = copy.deepcopy(rental_listings)


In [8]:
# Further extraction and pre-cleaning starting with sale_listings_copy
property_history = defaultdict(list)
property_agent = defaultdict(list)
property_officer = defaultdict(list)
property_sales_info = defaultdict(list)

for state, listings in sale_listings_copy.items():
    for listing in listings:
        # grab the unique id
        listing_id = listing["id"]

        # extract & inject id into history
        history = listing.pop("history", {})
        for date_key, event in history.items():
            event["id"] = listing_id

        # extract & inject id into agent
        listing_agent = listing.pop("listingAgent", {})
        if listing_agent:
            listing_agent["id"] = listing_id

        # extract & inject id into office
        listing_office = listing.pop("listingOffice", {})
        if listing_office:
            listing_office["id"] = listing_id

        # whatever remains is my “sales info” (it still has 'id' there)
        sales = listing

        # putting all together
        property_history[state].append(history)
        property_agent[state].append(listing_agent)
        property_officer[state].append(listing_office)
        property_sales_info[state].append(sales)


In [9]:
# Next is the rental_listings_copy

# Prepare collectors
rental_history = defaultdict(list)
rental_listing_info = defaultdict(list)

# Extract history + rest for each listing
for state, listings in rental_listings_copy.items():
    for listing in listings:
        listing_id = listing["id"]

        # pop & tag history
        history = listing.pop("history", {})
        for date_key, event in history.items():
            event["id"] = listing_id
        rental_history[state].append(history)

        # everything left is the “rest” of the listing
        #    (it still includes 'id' plus all other top-level fields)
        rest = listing
        rental_listing_info[state].append(rest)

#### Extraction of the data fields from each of the sales_listings data points

In [10]:
# Exacting the fields i want to keep in property_sales_info:
fields = [
    "id", "addressLine1", "city", "state", "zipCode", "county",
    "propertyType", "bedrooms", "bathrooms", "squareFootage",
    "lotSize", "yearBuilt", "price", "listedDate", "daysOnMarket"
]

# Preparing a flat list to collect each listing’s info
extracted_property_sales_info = []

for listings in property_sales_info.values():        # loop each state’s list
    for listing in listings:                  # loop each listing dict
        # Building a new dict with only the desired keys
        info = {}
        for field in fields:
            info[field] = listing.get(field)
        # Add it to the flat list
        extracted_property_sales_info.append(info)

In [11]:
# Exacting the fields i want to keep in property_history:
fields = [
    "event", "price", "listingType", "listedDate",
    "removedDate", "daysOnMarket", "id"
]

extracted_property_history = []

for state_list in property_history.values():        # loop each state’s list
    for date_dict in state_list:                  # loop each {"2025-06-25": {...}} entry
        # date_dict has one key (the date) and one value (the actual listing dict)
        for date_key, listing in date_dict.items():
            # building info dict
            info = {}
            for field in fields:
                # copy the field value (or None if missing)
                value = listing.get(field)
                info[field] = value
            extracted_property_history.append(info)


In [12]:
# Exacting the fields you want to keep in property_agent:
fields = ["name", "phone", "email", "id"]



# Preparing a flat list to collect each listing’s info
extracted_property_agent = []

for listings in property_agent.values():        # loop each state’s list
    for listing in listings:                  # loop each listing dict
        # Build a new dict with only the desired keys
        info = {}
        for field in fields:
            info[field] = listing.get(field)
        # Add it to the flat list
        extracted_property_agent.append(info)

In [13]:
# Exacting the fields i want to keep in property_officer:
fields = ["name", "phone", "email", "website", "id"]


# Preparing a flat list to collect each listing’s info
extracted_property_officer = []

for listings in property_officer.values():        # loop each state’s list
    for listing in listings:                  # loop each listing dict
        # Building a new dict with only the desired keys
        info = {}
        for field in fields:
            info[field] = listing.get(field)
        # Add it to the flat list
        extracted_property_officer.append(info)

#### Extraction of the data fields from each of the rental_listings data points

In [14]:
# Exacting the fields i want to keep in rental_listing_info:
fields = [
    "id", "addressLine1", "city", "state", "zipCode", "county",
    "propertyType", "bedrooms", "bathrooms", "squareFootage",
    "price", "listedDate", "daysOnMarket"
]


# Preparing a flat list to collect each listing’s info
extracted_rental_listing_info = []

for listings in rental_listing_info.values():        # loop each state’s list
    for listing in listings:                  # loop each listing dict
        # Building a new dict with only the desired keys
        info = {}
        for field in fields:
            info[field] = listing.get(field)
        # Add it to the flat list
        extracted_rental_listing_info.append(info)

In [15]:
# Exacting the fields i want to keep in rental_history:
fields = [
    "event", "price", "listingType", "listedDate",
    "removedDate", "daysOnMarket", "id"
]

extracted_rental_history = []

for state_list in rental_history.values():        # loop each state’s list
    for date_dict in state_list:                  # loop each {"2025-06-25": {...}} entry
        # date_dict has one key (the date) and one value (the actual listing dict)
        for date_key, listing in date_dict.items():
            # building info dict
            info = {}
            for field in fields:
                # copy the field value (or None if missing)
                value = listing.get(field)
                info[field] = value
            extracted_rental_history.append(info)

In [16]:
# print(json.dumps(extracted_property_history, indent=2))

#### Data points for rental_listings
extracted_rental_history and extracted_rental_listing_info


#### Data points for sale_listings
extracted_property_history, extracted_property_agent, extracted_property_officer and extracted_property_sales_info


#### Creating the dataframes for sales_listings data points

In [17]:
#### Creating the dataframes for sales_listings data points
property_history_df = pd.DataFrame(extracted_property_history)
property_agent_df = pd.DataFrame(extracted_property_agent)
property_officer_df = pd.DataFrame(extracted_property_officer)
property_sales_info_df = pd.DataFrame(extracted_property_sales_info)

##### ***Reordering property_history_df***

In [18]:
# ***Reordering property_history_df***

# Start with an empty column
property_history_df['sales_history_id'] = None

# Fill it row by row
for num in property_history_df.index:
    property_history_df.at[num, 'sales_history_id'] = str(uuid.uuid4())


In [19]:
# 1) Move history_id to be the very first column
property_history_df.insert(0, 'sales_history_id', property_history_df.pop('sales_history_id'))

# 2) Pop out the old “id” column and insert it as “sales_id” in position 1
property_history_df.insert(1, 'sales_id', property_history_df.pop('id'))


##### ***Reordering property_agent_df***

In [20]:
# ***Reordering property_agent_df***

# Start with an empty column
property_agent_df['agent_id'] = None

# Fill it row by row
for num in property_agent_df.index:
    property_agent_df.at[num, 'agent_id'] = str(uuid.uuid4())

In [21]:
# 1) Move history_id to be the very first column
property_agent_df.insert(0, 'agent_id', property_agent_df.pop('agent_id'))

# 2) Pop out the old “id” column and insert it as “sales_id” in position 1
property_agent_df.insert(1, 'sales_id', property_agent_df.pop('id'))

##### ***Reordering property_officer_df***

In [22]:
# ***Reordering property_officer_df***

# Start with an empty column
property_officer_df['officer_id'] = None

# Fill it row by row
for num in property_officer_df.index:
    property_officer_df.at[num, 'officer_id'] = str(uuid.uuid4())

In [23]:
# 1) Move history_id to be the very first column
property_officer_df.insert(0, 'officer_id', property_officer_df.pop('officer_id'))

# 2) Pop out the old “id” column and insert it as “sales_id” in position 1
property_officer_df.insert(1, 'sales_id', property_officer_df.pop('id'))

##### ***Reordering property_sales_info_df***

In [24]:
##### ***Reordering property_sales_info_df***

# Pop out the old “id” column and insert it as “sales_id” in position 1
property_sales_info_df.insert(0, 'sales_id', property_sales_info_df.pop('id'))

#### Creating the dataframes for rental_listings data points

In [25]:
#### Creating the dataframes for rental_listings data points
rental_history_df = pd.DataFrame(extracted_rental_history)
rental_listing_info_df = pd.DataFrame(extracted_rental_listing_info)

##### ***Reordering rental_history_df***

In [26]:
# ***Reordering rental_history_df***

# Start with an empty column
rental_history_df['rental_history_id'] = None

# Fill it row by row
for num in rental_history_df.index:
    rental_history_df.at[num, 'rental_history_id'] = str(uuid.uuid4())

In [27]:
# 1) Move history_id to be the very first column
rental_history_df.insert(0, 'rental_history_id', rental_history_df.pop('rental_history_id'))

# 2) Pop out the old “id” column and insert it as “sales_id” in position 1
rental_history_df.insert(1, 'rental_id', rental_history_df.pop('id'))

##### ***Reordering rental_listing_info_df***

In [28]:
##### ***Reordering rental_listing_info_df***

# Pop out the old “id” column and insert it as “sales_id” in position 1
rental_listing_info_df.insert(0, 'rental_id', rental_listing_info_df.pop('id'))

####  List of dataframes for sales_listings data points
property_history_df
property_agent_df
property_officer_df 
property_sales_info_df 


####  List of dataframes for rental_listings data points
rental_history_df
rental_listing_info_df

###### Cleaning procedure for property_history_df

In [29]:
# Cleaning procedure for property_history_df

# Drop multiple columns, returning a new DataFrame
property_history_df.drop(columns=["event", "listingType", "removedDate"], inplace=True)

In [30]:
# Dealing withd duplicates
property_history_df.drop_duplicates(subset=['sales_id'], keep='first')

,sales_history_id,sales_id,price,listedDate,daysOnMarket
0,29e19108-66dd-4109-9cb9-98fdeab4a204,"540-Marker-Ln,-Wellsburg,-WV-26070",315000.0,2025-06-22T00:00:00.000Z,4
1,26154ed6-d696-4c53-8b67-d6a8f73bb534,"206-Crystal-Ln,-Weirton,-WV-26062",184900.0,2025-06-23T00:00:00.000Z,3
2,6a1b3233-700a-4536-a62e-9a0413e0b7a8,"203-Park-Ave,-Middlebourne,-WV-26149",309900.0,2025-06-21T00:00:00.000Z,5
3,61d223b4-9ed2-432b-901e-79c246c49e10,"963-Ivydale-Rd,-Clay,-WV-25043",255000.0,2025-06-24T00:00:00.000Z,2
4,9ba082f4-a4bf-4143-ab46-b717143f7bdb,"13088-Clay-Hwy,-Lizemores,-WV-25125",235000.0,2024-04-23T00:00:00.000Z,185
...,...,...,...,...,...
4351,6bcdfa4b-2d73-446e-8024-399d6d86e479,"116-Dagsworthy-Ave,-Dewey-Beach,-DE-19971",875000.0,2025-06-23T00:00:00.000Z,2
4352,c4108e40-e147-4e49-a044-111bb7b9eb52,"108-Driftwood-Cir,-Milford,-DE-19963",59000.0,2025-06-21T00:00:00.000Z,4
4353,4463f760-3ced-405c-b0f7-4fdca09d4570,"150-Woodland-Ave,-Ocean-View,-DE-19970",199990.0,2025-06-22T00:00:00.000Z,3
4354,c22e4c18-7680-452d-867f-a7168f2488d1,"223-Chad-Place-Mnr,-Ocean-View,-DE-19970",199000.0,2025-06-22T00:00:00.000Z,3


In [31]:
# Dealing with datetime
property_history_df['listedDate']  = pd.to_datetime(property_history_df['listedDate'])

In [32]:
# Handling Numerics → integer or float
property_history_df['daysOnMarket'] = property_history_df['daysOnMarket'].astype('Int64')

# converting price to int by rounding off
property_history_df['price'] = property_history_df['price'].round(0).astype('Int64')

In [33]:
# Checking for null values
print(property_history_df.isnull().sum())

sales_history_id    0
sales_id            0
price               0
listedDate          0
daysOnMarket        0
dtype: int64


# Cleaning procedure for property_agent_df

In [34]:
# Cleaning procedure for property_agent_df

# Dropping any rows where sales_id is null
property_agent_df.dropna(subset=['sales_id'], inplace=True)

# Now property_agent_df has no rows where sales_id is null.

In [35]:
# Checking for null values
print(property_agent_df.isnull().sum())

agent_id      0
sales_id      0
name          0
phone        83
email       110
dtype: int64


In [36]:
# Filling the missing values
property_agent_df['phone'].fillna('000-000-0000', inplace=True)
property_agent_df['email'].fillna('unknown@example.com', inplace=True)

C:\Users\Nel\AppData\Local\Temp\ipykernel_18016\2481061705.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  property_agent_df['phone'].fillna('000-000-0000', inplace=True)
C:\Users\Nel\AppData\Local\Temp\ipykernel_18016\2481061705.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

In [37]:
# Trim whitespace & normalize casing

property_agent_df['name']  = property_agent_df['name'].str.strip().str.title()   # “erica tamburin” → “Erica Tamburin”  
property_agent_df['email'] = property_agent_df['email'].str.strip().str.lower()  # remove stray spaces, lowercase  

In [38]:
# Dealing withd duplicates
property_agent_df.drop_duplicates(subset=['sales_id'], keep='first')

,agent_id,sales_id,name,phone,email
0,bc4129d8-e215-413e-92fd-ad6768647a17,"540-Marker-Ln,-Wellsburg,-WV-26070",Erica Tamburin,3045593505,ericatamburin@gmail.com
1,ab5adacc-3a2c-45af-937a-e2c92a11f4a1,"206-Crystal-Ln,-Weirton,-WV-26062",Robert Gilbert,3047488400,robertgilbert@howardhanna.com
2,ee416b5b-3ca5-414b-ab0f-f90f0d1fdf7e,"203-Park-Ave,-Middlebourne,-WV-26149",Brenda Riggs,3046657740,riggsbj@hotmail.com
3,a9224c67-77ff-456a-a6ba-5d136d009f4d,"963-Ivydale-Rd,-Clay,-WV-25043",Levi Morris,3044884174,morrislevi064@gmail.com
4,f9d6487b-1248-4a94-9970-5c09dcb27523,"13088-Clay-Hwy,-Lizemores,-WV-25125",Levi Morris,3044884174,morrislevi064@gmail.com
...,...,...,...,...,...
3537,da8cf2d6-3c61-4876-b2fb-39a0b1279307,"116-Dagsworthy-Ave,-Dewey-Beach,-DE-19971",Chad Mitchell,3023788700,realinfo@psre.com
3538,9452cb85-aeda-4b40-b77d-ab1c3535772f,"108-Driftwood-Cir,-Milford,-DE-19963",Renee Edge,3025355599,reneeedge@kw.com
3539,a67bb408-b85a-4d2c-8404-67939797f696,"150-Woodland-Ave,-Ocean-View,-DE-19970",Mike Hall,4433734046,mike@bluecoastalgroup.com
3540,9964b492-0c50-4c55-970c-ad46c9e749be,"223-Chad-Place-Mnr,-Ocean-View,-DE-19970","Brittingham, Gary",3027455605,g.brittingham@mchsi.com


# Cleaning procedure for property_officer_df

In [39]:
# Cleaning procedure for property_officer_df

# Dropping any rows where sales_id is null
property_officer_df.dropna(subset=['sales_id'], inplace=True)

# Now property_agent_df has no rows where sales_id is null.

In [40]:
# Checking for null values
print(property_officer_df.isnull().sum())

officer_id       0
sales_id         0
name             0
phone           29
email          298
website       1052
dtype: int64


In [41]:
# Drop redundant columns returning
property_officer_df.drop(columns=["website"], inplace=True)

In [42]:
# Filling the missing values
property_officer_df['phone'].fillna('000-000-0000', inplace=True)
property_officer_df['email'].fillna('unknown@example.com', inplace=True)

C:\Users\Nel\AppData\Local\Temp\ipykernel_18016\2090476851.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  property_officer_df['phone'].fillna('000-000-0000', inplace=True)
C:\Users\Nel\AppData\Local\Temp\ipykernel_18016\2090476851.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [43]:
# Trim whitespace & normalize casing
property_officer_df['name']  = property_officer_df['name'].str.strip().str.title()   # “erica tamburin” → “Erica Tamburin”  
property_officer_df['email'] = property_officer_df['email'].str.strip().str.lower()  # remove stray spaces, lowercase  

In [44]:
# Dealing withd duplicates
property_officer_df.drop_duplicates(subset=['sales_id'], keep='first')

,officer_id,sales_id,name,phone,email
0,6ad8e5de-d33c-47a0-b0fc-2a8d62361492,"540-Marker-Ln,-Wellsburg,-WV-26070",Re/Max Broadwater,3042422600,travisbroadwater@gmail.com
1,1f16ba85-c06d-4551-aa7b-aab378bab0ab,"206-Crystal-Ln,-Weirton,-WV-26062",Howard Hanna - Weirton - Mortimer Realty Compa...,8776617587,manager.weirton@howardhanna.com
2,975411af-7630-4e39-b7d8-959bbede783a,"203-Park-Ave,-Middlebourne,-WV-26149","Riggs Realty, Llc.",3046657740,riggsbj@hotmail.com
3,0d9898ba-a944-4d86-a91c-4c60b00fe7fe,"963-Ivydale-Rd,-Clay,-WV-25043","Lpt Realty, Llc",8773662213,wvbrokers@lptrealty.com
4,05d5c926-0044-43b2-866a-a03ddd653d19,"13088-Clay-Hwy,-Lizemores,-WV-25125","Lpt Realty, Llc",8773662213,wvbrokers@lptrealty.com
...,...,...,...,...,...
3537,d269a299-ec50-42a6-85c3-433d45520d90,"116-Dagsworthy-Ave,-Dewey-Beach,-DE-19971",Crown Homes Real Estate,3025046147,broker@forakersales.com
3538,770443ea-e88c-447a-960c-85f18b7cf006,"108-Driftwood-Cir,-Milford,-DE-19963",Keller Williams Realty,3023600300,walt.t@kw.com
3539,79b983fd-9748-4f97-920f-c10cefc495b9,"150-Woodland-Ave,-Ocean-View,-DE-19970",Patterson-Schwartz Ocean View Center,3025371300,realinfo@psre.com
3540,71d4b15b-0f26-44be-bb0b-3e7c00c7d4c8,"223-Chad-Place-Mnr,-Ocean-View,-DE-19970",Jack Lingo Millsboro,8889343970,millsboro@mchsi.com


# Cleaning procedure for property_sales_info_df

In [45]:
# Cleaning procedure for property_sales_info_df

# Converting to the right data type for strings and categorise
property_sales_info_df['sales_id'] = property_sales_info_df['sales_id'].astype(str)
property_sales_info_df['addressLine1'] = property_sales_info_df['addressLine1'].astype(str)
property_sales_info_df['zipCode'] = property_sales_info_df['zipCode'].astype(str)
property_sales_info_df['propertyType'] = property_sales_info_df['propertyType'].astype('category')

In [46]:
# Checking for null values
print(property_sales_info_df.isnull().sum())

sales_id            0
addressLine1        0
city                0
state               0
zipCode             0
county              0
propertyType        0
bedrooms          157
bathrooms         172
squareFootage     223
lotSize          1223
yearBuilt         718
price               0
listedDate          0
daysOnMarket        0
dtype: int64


In [47]:
# Drop redundant columns returning
property_sales_info_df.drop(columns=["lotSize"], inplace=True)

In [48]:
# Convert your float‐with‐NaNs column into a nullable integer column:
property_sales_info_df['bedrooms'] = property_sales_info_df['bedrooms'].astype('Int64')
property_sales_info_df['squareFootage'] = property_sales_info_df['squareFootage'].astype('Int64')
property_sales_info_df['daysOnMarket'] = property_sales_info_df['daysOnMarket'].astype('Int64')
property_sales_info_df['yearBuilt'] = property_sales_info_df['yearBuilt'].astype('Int64')

# converting price to int by rounding off
property_sales_info_df['price'] = property_sales_info_df['price'].round(0).astype('Int64')


In [49]:
# Dealing with datetime
property_sales_info_df['listedDate'] = pd.to_datetime(property_sales_info_df['listedDate'], utc=True, errors='coerce')

In [50]:
# Address normalization

# Strip leading/trailing spaces
property_sales_info_df['addressLine1'] = property_sales_info_df['addressLine1'].str.strip()

# Collapse multiple internal spaces
property_sales_info_df['addressLine1'] = property_sales_info_df['addressLine1'].str.replace(r'\s+', ' ', regex=True)

# Title-case (but preserve acronyms like “US-1”)
property_sales_info_df['addressLine1'] = property_sales_info_df['addressLine1'].str.title()


In [51]:
# Dealing withd duplicates
property_sales_info_df.drop_duplicates(subset=['sales_id'], keep='first')

,sales_id,addressLine1,city,state,zipCode,county,propertyType,bedrooms,bathrooms,squareFootage,yearBuilt,price,listedDate,daysOnMarket
0,"540-Marker-Ln,-Wellsburg,-WV-26070",540 Marker Ln,Wellsburg,WV,26070,Brooke,Single Family,4,2.5,1800,1975,315000,2025-06-22 00:00:00+00:00,4
1,"206-Crystal-Ln,-Weirton,-WV-26062",206 Crystal Ln,Weirton,WV,26062,Brooke,Single Family,2,1.0,1172,1957,184900,2025-06-23 00:00:00+00:00,3
2,"203-Park-Ave,-Middlebourne,-WV-26149",203 Park Ave,Middlebourne,WV,26149,Tyler,Single Family,3,2.0,2348,1964,309900,2025-06-21 00:00:00+00:00,5
3,"963-Ivydale-Rd,-Clay,-WV-25043",963 Ivydale Rd,Clay,WV,25043,Clay,Single Family,3,2.0,2700,1986,255000,2025-06-24 00:00:00+00:00,2
4,"13088-Clay-Hwy,-Lizemores,-WV-25125",13088 Clay Hwy,Lizemores,WV,25125,Clay,Single Family,3,2.0,2100,1940,220000,2025-06-24 00:00:00+00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,"116-Dagsworthy-Ave,-Dewey-Beach,-DE-19971",116 Dagsworthy Ave,Dewey Beach,DE,19971,Sussex,Single Family,2,1.0,<NA>,1972,875000,2025-06-23 00:00:00+00:00,2
3538,"108-Driftwood-Cir,-Milford,-DE-19963",108 Driftwood Cir,Milford,DE,19963,Sussex,Land,<NA>,NaN,<NA>,<NA>,59000,2025-06-21 00:00:00+00:00,4
3539,"150-Woodland-Ave,-Ocean-View,-DE-19970",150 Woodland Ave,Ocean View,DE,19970,Sussex,Land,<NA>,NaN,<NA>,<NA>,199990,2025-06-22 00:00:00+00:00,3
3540,"223-Chad-Place-Mnr,-Ocean-View,-DE-19970",223 Chad Place Mnr,Ocean View,DE,19970,Sussex,Land,<NA>,NaN,<NA>,<NA>,199000,2025-06-22 00:00:00+00:00,3


###### Cleaning procedure for rental_history_df

In [52]:
# Cleaning procedure for rental_history_df

# Drop multiple columns, returning a new DataFrame
rental_history_df.drop(columns=["event", "listingType", "removedDate"], inplace=True)

In [53]:
# Dealing withd duplicates
rental_history_df.drop_duplicates(subset=['rental_id'], keep='first')

,rental_history_id,rental_id,price,listedDate,daysOnMarket
0,6852682b-aa48-4204-8b0b-b98fec8859b7,"127-Denham-St,-Clarksburg,-WV-26301",1100,2025-06-25T00:00:00.000Z,1
1,bc227a12-f285-4d3d-bbc3-df76d620db99,"614-Hart-St,-Clarksburg,-WV-26301",1250,2025-06-25T00:00:00.000Z,1
2,3e55e329-668d-4947-926e-f98e6261f9a9,"772-Limestone-Run-Rd,-Clarksburg,-WV-26301",1400,2025-04-24T00:00:00.000Z,1
4,ef3e053c-6c07-4711-9cc4-9e7dc567f677,"104-Sassafras-Way,-Clarksburg,-WV-26301",2200,2025-06-25T00:00:00.000Z,1
5,bd567065-4166-41f0-b7ec-616274b5403a,"204-1-2-Burton-Ave,-Nutter-Fort,-WV-26301",850,2025-06-24T00:00:00.000Z,2
...,...,...,...,...,...
7805,6824479f-fb98-47c2-abea-42336411c6be,"1-Allandale-Dr,---D17,-Newark,-DE-19713",1397,2025-06-14T00:00:00.000Z,1
7807,c1dbdded-98a7-4d0d-a403-33c46e66e307,"1314-Wharton-Dr,---610,-Newark,-DE-19711",1755,2024-10-30T00:00:00.000Z,1
7810,a09f2cb0-a57b-4745-bc0e-b78b47a5e9b4,"735-Wollaston-Ave,---Full,-Newark,-DE-19711",3100,2025-06-23T00:00:00.000Z,1
7811,c386ec7c-b93d-4d8e-81d2-efd2ca0b406b,"50-Sandalwood-Dr,---2407,-Newark,-DE-19713",1697,2025-06-23T00:00:00.000Z,1


In [54]:
# Dealing with datetime
rental_history_df['listedDate']  = pd.to_datetime(rental_history_df['listedDate'])

In [55]:
# Handling Numerics → integer or float
rental_history_df['price'] = rental_history_df['price'].astype('Int64')
rental_history_df['daysOnMarket'] = rental_history_df['daysOnMarket'].astype('Int64')


In [56]:
# Checking for null values
print(rental_history_df.isnull().sum())

rental_history_id    0
rental_id            0
price                0
listedDate           0
daysOnMarket         0
dtype: int64


# Cleaning procedure for rental_listing_info_df

In [57]:
# Cleaning procedure for rental_listing_info_df

# Converting to the right data type for strings and categorise
rental_listing_info_df['rental_id'] = rental_listing_info_df['rental_id'].astype(str)
rental_listing_info_df['addressLine1'] = rental_listing_info_df['addressLine1'].astype(str)
rental_listing_info_df['zipCode'] = rental_listing_info_df['zipCode'].astype(str)
rental_listing_info_df['propertyType'] = rental_listing_info_df['propertyType'].astype('category')

In [58]:
# Checking for null values
print(rental_listing_info_df.isnull().sum())

rental_id          0
addressLine1       0
city               0
state              0
zipCode            0
county             0
propertyType       0
bedrooms           2
bathrooms          6
squareFootage    330
price              0
listedDate         0
daysOnMarket       0
dtype: int64


In [59]:
# Convert your float‐with‐NaNs column into a nullable integer column:
rental_listing_info_df['bedrooms'] = rental_listing_info_df['bedrooms'].astype('Int64')
rental_listing_info_df['squareFootage'] = rental_listing_info_df['squareFootage'].astype('Int64')
rental_listing_info_df['daysOnMarket'] = rental_listing_info_df['daysOnMarket'].astype('Int64')

# converting price to int by rounding off
rental_listing_info_df['price'] = rental_listing_info_df['price'].round(0).astype('Int64')


In [60]:
# Dealing with datetime
property_sales_info_df['listedDate'] = pd.to_datetime(property_sales_info_df['listedDate'], utc=True, errors='coerce')

In [61]:
# Address normalization

# Strip leading/trailing spaces
rental_listing_info_df['addressLine1'] = rental_listing_info_df['addressLine1'].str.strip()

# Collapse multiple internal spaces
rental_listing_info_df['addressLine1'] = rental_listing_info_df['addressLine1'].str.replace(r'\s+', ' ', regex=True)

# Title-case (but preserve acronyms like “US-1”)
rental_listing_info_df['addressLine1'] = rental_listing_info_df['addressLine1'].str.title()


In [62]:
# Dealing withd duplicates
rental_listing_info_df.drop_duplicates(subset=['rental_id'], keep='first')

,rental_id,addressLine1,city,state,zipCode,county,propertyType,bedrooms,bathrooms,squareFootage,price,listedDate,daysOnMarket
0,"127-Denham-St,-Clarksburg,-WV-26301",127 Denham St,Clarksburg,WV,26301,Harrison,Single Family,2,1.0,936,1100,2025-06-25T00:00:00.000Z,1
1,"614-Hart-St,-Clarksburg,-WV-26301",614 Hart St,Clarksburg,WV,26301,Harrison,Single Family,2,1.0,893,1250,2025-06-25T00:00:00.000Z,1
2,"772-Limestone-Run-Rd,-Clarksburg,-WV-26301",772 Limestone Run Rd,Clarksburg,WV,26301,Harrison,Apartment,2,1.0,700,950,2025-06-25T00:00:00.000Z,1
3,"104-Sassafras-Way,-Clarksburg,-WV-26301",104 Sassafras Way,Clarksburg,WV,26301,Harrison,Single Family,3,3.0,1712,2200,2025-06-25T00:00:00.000Z,1
4,"204-1-2-Burton-Ave,-Nutter-Fort,-WV-26301",204 1/2 Burton Ave,Nutter Fort,WV,26301,Harrison,Single Family,2,1.0,<NA>,850,2025-06-24T00:00:00.000Z,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,"1-Allandale-Dr,---D17,-Newark,-DE-19713",1 Allandale Dr,Newark,DE,19713,New Castle,Apartment,1,1.0,630,1397,2025-06-23T00:00:00.000Z,1
3412,"1314-Wharton-Dr,---610,-Newark,-DE-19711",1314 Wharton Dr,Newark,DE,19711,New Castle,Apartment,2,2.0,950,1780,2025-06-23T00:00:00.000Z,1
3413,"735-Wollaston-Ave,---Full,-Newark,-DE-19711",735 Wollaston Ave,Newark,DE,19711,New Castle,Townhouse,4,2.0,2225,3100,2025-06-23T00:00:00.000Z,1
3414,"50-Sandalwood-Dr,---2407,-Newark,-DE-19713",50 Sandalwood Dr,Newark,DE,19713,New Castle,Apartment,2,1.0,925,1697,2025-06-23T00:00:00.000Z,1


In [74]:
property_sales_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   sales_id       3542 non-null   object             
 1   addressLine1   3542 non-null   object             
 2   city           3542 non-null   object             
 3   state          3542 non-null   object             
 4   zipCode        3542 non-null   object             
 5   county         3542 non-null   object             
 6   propertyType   3542 non-null   category           
 7   bedrooms       3385 non-null   Int64              
 8   bathrooms      3370 non-null   float64            
 9   squareFootage  3319 non-null   Int64              
 10  yearBuilt      2824 non-null   Int64              
 11  price          3542 non-null   Int64              
 12  listedDate     3542 non-null   datetime64[ns, UTC]
 13  daysOnMarket   3542 non-null   Int64            

In [76]:
# Last minute cleaning

# State code we want to replace using lookup dict
lookup = {
    'WV': 'West Virginia',
    'VA': 'Virginia',
    'SC': 'South Carolina',
    'NC': 'North Carolina',
    'MD': 'Maryland',
    'GA': 'Georgia',
    'FL': 'Florida',
    'DE': 'Delaware'
}


# overwrite the code column in-place
property_sales_info_df['state'] = property_sales_info_df['state'].replace(lookup)

# rename addressLine1 to address
property_sales_info_df.rename(columns={'addressLine1': 'address'}, inplace=True)


property_sales_info_df

,sales_id,address,city,state,zipCode,county,propertyType,bedrooms,bathrooms,squareFootage,yearBuilt,price,listedDate,daysOnMarket
0,"540-Marker-Ln,-Wellsburg,-WV-26070",540 Marker Ln,Wellsburg,West Virginia,26070,Brooke,Single Family,4,2.5,1800,1975,315000,2025-06-22 00:00:00+00:00,4
1,"206-Crystal-Ln,-Weirton,-WV-26062",206 Crystal Ln,Weirton,West Virginia,26062,Brooke,Single Family,2,1.0,1172,1957,184900,2025-06-23 00:00:00+00:00,3
2,"203-Park-Ave,-Middlebourne,-WV-26149",203 Park Ave,Middlebourne,West Virginia,26149,Tyler,Single Family,3,2.0,2348,1964,309900,2025-06-21 00:00:00+00:00,5
3,"963-Ivydale-Rd,-Clay,-WV-25043",963 Ivydale Rd,Clay,West Virginia,25043,Clay,Single Family,3,2.0,2700,1986,255000,2025-06-24 00:00:00+00:00,2
4,"13088-Clay-Hwy,-Lizemores,-WV-25125",13088 Clay Hwy,Lizemores,West Virginia,25125,Clay,Single Family,3,2.0,2100,1940,220000,2025-06-24 00:00:00+00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,"116-Dagsworthy-Ave,-Dewey-Beach,-DE-19971",116 Dagsworthy Ave,Dewey Beach,Delaware,19971,Sussex,Single Family,2,1.0,<NA>,1972,875000,2025-06-23 00:00:00+00:00,2
3538,"108-Driftwood-Cir,-Milford,-DE-19963",108 Driftwood Cir,Milford,Delaware,19963,Sussex,Land,<NA>,NaN,<NA>,<NA>,59000,2025-06-21 00:00:00+00:00,4
3539,"150-Woodland-Ave,-Ocean-View,-DE-19970",150 Woodland Ave,Ocean View,Delaware,19970,Sussex,Land,<NA>,NaN,<NA>,<NA>,199990,2025-06-22 00:00:00+00:00,3
3540,"223-Chad-Place-Mnr,-Ocean-View,-DE-19970",223 Chad Place Mnr,Ocean View,Delaware,19970,Sussex,Land,<NA>,NaN,<NA>,<NA>,199000,2025-06-22 00:00:00+00:00,3


In [ ]:
# Last minute cleaning for rental_listing_info_df
# State code we want to replace using lookup dict
lookup = {
    'WV': 'West Virginia',
    'VA': 'Virginia',
    'SC': 'South Carolina',
    'NC': 'North Carolina',
    'MD': 'Maryland',
    'GA': 'Georgia',
    'FL': 'Florida',
    'DE': 'Delaware'
}
# overwrite the code column in-place
rental_listing_info_df['state'] = rental_listing_info_df['state'].replace(lookup)

# rename addressLine1 to address
rental_listing_info_df.rename(columns={'addressLine1': 'address'}, inplace=True)

rental_listing_info_df

,rental_id,address,city,state,zipCode,county,propertyType,bedrooms,bathrooms,squareFootage,price,listedDate,daysOnMarket
0,"127-Denham-St,-Clarksburg,-WV-26301",127 Denham St,Clarksburg,West Virginia,26301,Harrison,Single Family,2,1.0,936,1100,2025-06-25T00:00:00.000Z,1
1,"614-Hart-St,-Clarksburg,-WV-26301",614 Hart St,Clarksburg,West Virginia,26301,Harrison,Single Family,2,1.0,893,1250,2025-06-25T00:00:00.000Z,1
2,"772-Limestone-Run-Rd,-Clarksburg,-WV-26301",772 Limestone Run Rd,Clarksburg,West Virginia,26301,Harrison,Apartment,2,1.0,700,950,2025-06-25T00:00:00.000Z,1
3,"104-Sassafras-Way,-Clarksburg,-WV-26301",104 Sassafras Way,Clarksburg,West Virginia,26301,Harrison,Single Family,3,3.0,1712,2200,2025-06-25T00:00:00.000Z,1
4,"204-1-2-Burton-Ave,-Nutter-Fort,-WV-26301",204 1/2 Burton Ave,Nutter Fort,West Virginia,26301,Harrison,Single Family,2,1.0,<NA>,850,2025-06-24T00:00:00.000Z,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,"1-Allandale-Dr,---D17,-Newark,-DE-19713",1 Allandale Dr,Newark,Delaware,19713,New Castle,Apartment,1,1.0,630,1397,2025-06-23T00:00:00.000Z,1
3412,"1314-Wharton-Dr,---610,-Newark,-DE-19711",1314 Wharton Dr,Newark,Delaware,19711,New Castle,Apartment,2,2.0,950,1780,2025-06-23T00:00:00.000Z,1
3413,"735-Wollaston-Ave,---Full,-Newark,-DE-19711",735 Wollaston Ave,Newark,Delaware,19711,New Castle,Townhouse,4,2.0,2225,3100,2025-06-23T00:00:00.000Z,1
3414,"50-Sandalwood-Dr,---2407,-Newark,-DE-19713",50 Sandalwood Dr,Newark,Delaware,19713,New Castle,Apartment,2,1.0,925,1697,2025-06-23T00:00:00.000Z,1


In [ ]:

# map each to a sheet name
dfs = {
    "sales_info":property_sales_info_df,
    "sales_history":property_history_df,
    "sales_agent":property_agent_df,
    "sales_officer":property_officer_df,
    "rental_history":rental_history_df,
    "rental_info":rental_listing_info_df,
}


# map each to a sheet name
dfs = {
    "sales_info":property_sales_info_df,
    "sales_history":property_history_df,
    "sales_agent":property_agent_df,
    "sales_officer":property_officer_df,
    "rental_history":rental_history_df,
    "rental_info":rental_listing_info_df,
}

# # write all your dfs to separate Parquet files
# for name, df in dfs.items():
#     df.to_parquet(f'{name}.parquet', engine='pyarrow', index=False)


with pd.ExcelWriter('Real_cast_sheets.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)


ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

####  List of dataframes for sales_listings data points
property_history_df
property_agent_df
property_officer_df 
property_sales_info_df 


####  List of dataframes for rental_listings data points
rental_history_df
rental_listing_info_df


The schema would be a relational 3NF schema.
This isn’t a dimensional (star or snowflake) schema at all but rather a classic normalized OLTP (relational) design—specifically laid out to meet 3rd Normal Form:

Each table represents a single entity (Property, Officer, Agent, History).

No redundancy: non-key attributes depend only on the key.

All relationships (1-to-many) are handled via foreign keys.

In other words, it’s simply a 3NF relational schema (sometimes just called a “normalized schema” or “transactional schema”), optimized for insert/update/delete consistency rather than for the aggregated-query performance you’d get from a star or snowflake.



Why this structure?
property_sales_info is the “master” table; its id (your existing primary key) uniquely identifies each property.

property_officer and property_agent each get their own surrogate PK (officer_id/agent_id) so you can store multiple officers or agents per property if needed (and track them individually).

property_history uses its own history_id PK, plus the property_id FK back to property_sales_info. This lets you record an arbitrary number of events (listings, rentals, sales) over time.

All foreign keys use ON DELETE CASCADE so that if you ever remove a property from property_sales_info, its associated officers, agents, and history entries are cleaned up automatically.

Feel free to tweak column types (e.g. use DATE instead of TIMESTAMP if you never need time-of-day) or add indexes (e.g. on listed_date) depending on your query patterns.

#### Loading the data into postgres

In [64]:
# Load your .env
load_dotenv()  # looks for a .env file in cwd

DB_USER     = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST     = os.getenv("DB_HOST", "localhost")
DB_PORT     = os.getenv("DB_PORT", "5432")
DB_NAME     = os.getenv("DB_NAME")

# Create the engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Prepare your DataFrames and target table names
tables = {
    "sales_info":property_sales_info_df,
    "sales_history":property_history_df,
    "sales_agent":property_agent_df,
    "sales_officer":property_officer_df,
    "rental_history":rental_history_df,
    "rental_info":rental_listing_info_df
}


# Loop and write each one
for table_name, df in tables.items():
    df.to_sql(
        name=table_name,
        con=engine,
        schema="public",         # adjust if you use another schema
        if_exists="replace",     # or "append" There are risk to using append for future batch jobs, like if it appends PK i already have in there, it'll break.
        index=False,             # drop the DataFrame’s index column
        chunksize=500            # adjust batch size to manage memory/performance
    )
    print(f"→ Loaded {len(df)} rows into {table_name}")



→ Loaded 3542 rows into sales_info
→ Loaded 4357 rows into sales_history
→ Loaded 2190 rows into sales_agent
→ Loaded 2218 rows into sales_officer
→ Loaded 7813 rows into rental_history
→ Loaded 3416 rows into rental_info


In [ ]:
# # Prepare your DataFrames and target table names
# tables = {
#     "sales_info":property_sales_info_df,
#     "sales_history":property_history_df,
#     "sales_agent":property_agent_df,
#     "sales_officer":property_officer_df,
#     "rental_history":rental_history_df,
#     "rental_info":rental_listing_info_df
# }

In [72]:
rental_history_df

,rental_history_id,rental_id,price,listedDate,daysOnMarket
0,6852682b-aa48-4204-8b0b-b98fec8859b7,"127-Denham-St,-Clarksburg,-WV-26301",1100,2025-06-25 00:00:00+00:00,1
1,bc227a12-f285-4d3d-bbc3-df76d620db99,"614-Hart-St,-Clarksburg,-WV-26301",1250,2025-06-25 00:00:00+00:00,1
2,3e55e329-668d-4947-926e-f98e6261f9a9,"772-Limestone-Run-Rd,-Clarksburg,-WV-26301",1400,2025-04-24 00:00:00+00:00,1
3,603fb771-2adf-433b-bc7d-fa45cc5cf355,"772-Limestone-Run-Rd,-Clarksburg,-WV-26301",950,2025-06-25 00:00:00+00:00,1
4,ef3e053c-6c07-4711-9cc4-9e7dc567f677,"104-Sassafras-Way,-Clarksburg,-WV-26301",2200,2025-06-25 00:00:00+00:00,1
...,...,...,...,...,...
7808,ff0df920-9f5e-402a-9e14-7fbed98378ae,"1314-Wharton-Dr,---610,-Newark,-DE-19711",1680,2025-04-15 00:00:00+00:00,1
7809,5f4a2498-a588-4dab-b847-477abbd4387b,"1314-Wharton-Dr,---610,-Newark,-DE-19711",1780,2025-06-23 00:00:00+00:00,1
7810,a09f2cb0-a57b-4745-bc0e-b78b47a5e9b4,"735-Wollaston-Ave,---Full,-Newark,-DE-19711",3100,2025-06-23 00:00:00+00:00,1
7811,c386ec7c-b93d-4d8e-81d2-efd2ca0b406b,"50-Sandalwood-Dr,---2407,-Newark,-DE-19713",1697,2025-06-23 00:00:00+00:00,1
